<a href="https://colab.research.google.com/github/cbrumm/deep-learning-with-python-book/blob/main/Chapter_2_MNIST_digits_with_tensorflow_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

class NaiveDense:
  """Represents a densely connected layer of a network."""


  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    b_initial_value = tf.zeros(output_size)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs):
    """Forward pass computing activation."""
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self):
    return [self.W, self.b]

class NaiveSequential:
  """Models a sequence of network layers."""

  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    """Implements the forward pass, calling layers in order."""
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      # concatenate weight lists
      weights += layer.weights
    return weights

In [5]:
model = NaiveSequential([
  NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
  NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4, "expecting 2 layers with 2 weights (W and b)"

In [6]:
class BatchGenerator:
  """Generates batches of data for training."""

  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels), "Need same number of tensors and labels"
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = len(images) // batch_size

  def next(self):
    """Returns the next batch of tenors and labels."""
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

def one_training_step(model, images_batch, labels_batch):
  """Performs a single training step."""
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
        labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)

  # Gradient of model.weights with respect to average_loss.
  gradients = tape.gradient(average_loss, model.weights)
  # Update model.weights in the direction indicated by gradient.
  update_weights(gradients, model.weights)
  return average_loss


learning_rate = 1e-3


# Ordinarily, we would use a Keras optimizer instance for weight updates.
def update_weights(gradients, weights):
  for gradient, weight in zip(gradients, weights):
    # Substract from tensorflow variable.
    weight.assign_sub(gradient * learning_rate)


def fit(model, images, labels, epochs, batch_size=128):
  """Run |epochs| training steps going over the data batches for each epoch."""
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    batch_generator = BatchGenerator(images, labels, batch_size)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"loss at batch {batch_counter}: {loss:.2f}")

In [7]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 0
loss at batch 0: 5.33
loss at batch 100: 2.24
loss at batch 200: 2.20
loss at batch 300: 2.06
loss at batch 400: 2.22
Epoch 1
loss at batch 0: 1.90
loss at batch 100: 1.88
loss at batch 200: 1.83
loss at batch 300: 1.69
loss at batch 400: 1.84
Epoch 2
loss at batch 0: 1.56
loss at batch 100: 1.58
loss at batch 200: 1.51
loss at batch 300: 1.40
loss at batch 400: 1.53
Epoch 3
loss at batch 0: 1.29
loss at batch 100: 1.34
loss at batch 200: 1.24
loss at batch 300: 1.19
loss at batch 400: 1.29
Epoch 4
loss at batch 0: 1.08
loss at batch 100: 1.16
loss at batch 200: 1.04
loss at batch 300: 1.03
loss at batch 400: 1.13
Epoch 5
loss at batch 0: 0.94
loss at batch 100: 1.02
loss at batch 200: 0.90
loss at batch 300: 0.92
loss at batch 400: 1.00
Epoch 6
loss at batch 0: 0.83
loss at batch 100: 0.91
loss at batch 200: 0.80
loss at batch 300: 0.83
loss at batch 400: 0.91
Epoch 7
loss at batch 0: 0.75
loss at batch 100: 0.83

In [8]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.83
